In [1]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from random import shuffle

In [2]:
image_size=53
channels=3
train_ratio=0.8
val_ratio=0.1

In [3]:
label = 0
train = []
val = []
test = []
folderpath="/home/justdial/cnn/items"
for cat in os.listdir(folderpath):
    print("label for category {} is {}".format(cat,label))
    images = os.listdir(os.path.join(folderpath,cat))
    totalimages = len(images)
    num_train = int(totalimages*train_ratio)
    num_val = int(totalimages*val_ratio)
    num_test = totalimages - (num_train + num_val)
    
    print(num_train)
    print(num_val)
    print(num_test)
    
    for i,img in enumerate(images):
        if(i<=num_train):
            train.append([os.path.join(os.path.join(folderpath,cat),img),label])
        elif(i>num_train and i<num_train+num_val):
             val.append([os.path.join(os.path.join(folderpath,cat),img),label])
        elif(i>=num_train+num_val and i<=totalimages):
             test.append([os.path.join(os.path.join(folderpath,cat),img),label])
    label+=1

print("total train =", len(train))
print("total val =", len(val))
print("total test =", len(test))

label for category reception is 0
306
38
39
label for category conference is 1
174
21
23
total train = 482
total val = 57
total test = 62


In [4]:
tfrecords_folder_path="/home/justdial/cnn/tfrecords"
writer1=tf.python_io.TFRecordWriter(tfrecords_folder_path + "/tfrecordtrain" )
writer2=tf.python_io.TFRecordWriter(tfrecords_folder_path + "/tfrecordval" )
writer3=tf.python_io.TFRecordWriter(tfrecords_folder_path + "/tfrecordtest" )

In [5]:
def wrap_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def wrap_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [6]:
def get_binary(image):
    img = Image.open(image)
    img = img.resize((image_size,image_size),Image.ANTIALIAS)
    img = img.tobytes()
    
    if(len(img) == image_size*image_size*channels):
        return img
    else:
        print("image not converted {}".format(image))

In [7]:
def create_tfrecord(img_bytes,label):
    data = {
        'image' : wrap_bytes(img_bytes),
        'label' : wrap_int64(label)}
    feature = tf.train.Features(feature=data)
    example = tf.train.Example(features=feature)
    serialized = example.SerializeToString()
    return serialized
    

In [8]:
for i in range(100):
    shuffle(train)
    shuffle(val)
    shuffle(test)

try: 
    for i in train:
        image = i[0]
        label = i[1]
        img_bytes = get_binary(image)
        serialized = create_tfrecord(img_bytes,label)
        writer1.write(serialized)
except:
    pass

try:
    for i in val:
        image = i[0]
        label = i[1]
        img_bytes = get_binary(image)
        serialized = create_tfrecord(img_bytes,label)
        writer2.write(serialized)
except:
    pass

try:
    for i in test:
        image = i[0]
        label = i[1]
        img_bytes = get_binary(image)
        serialized = create_tfrecord(img_bytes,label)
        writer3.write(serialized)
except:
    pass
        
writer1.close()
writer2.close()
writer3.close()

image not converted /home/justdial/cnn/items/reception/21053888.png
image not converted /home/justdial/cnn/items/conference/51203355.png


In [9]:
def parse(serialized):
    features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([],tf.int64)
    }
    parsed_example=tf.parse_single_example(serialized=serialized,
                                          features=features)
    raw_image=parsed_example['image']
    image=tf.decode_raw(raw_image,tf.uint8)
    
    image = tf.reshape(image,shape=[image_size,image_size,channels])

    image=tf.cast(image,tf.float64)    
    label=parsed_example['label']
    return image,label

In [ ]:
# # def load_tfrecord:
    
# traindataset=tf.data.TFRecordDataset(filenames='/home/justdial/cnn/tfrecords/tfrecordtrain')
# traindataset=traindataset.map(parse)
# # print((traindataset))

# valdataset=tf.data.TFRecordDataset(filenames='/home/justdial/cnn/tfrecords/tfrecordval')
# valdataset=valdataset.map(parse)
# #print(len(valdataset))

# testdataset=tf.data.TFRecordDataset(filenames='/home/justdial/cnn/tfrecords/tfrecordtest')
# testdataset=testdataset.map(parse)
# #print(len(testdataset))

In [10]:
def input_fn(filenames,train, batch_size=32,buffer_size=2048):
    
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    dataset = dataset.map(parse)
    if train:
        dataset=dataset.shuffle(buffer_size=buffer_size)
        num_repeat=None
    else:
        num_repeat=1
    dataset = dataset.repeat(num_repeat)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    img_batch,Y = iterator.get_next()
    X={'image':img_batch}

    return X,Y

In [ ]:
# iterator= tf.data.Iterator.from_structure(output_types = traindataset.output_types, 
#                                           output_shapes = traindataset.output_shapes)

In [ ]:
# train_init = iterator.make_initializer(traindataset)
# val_init = iterator.make_initializer(valdataset)
# test_init = iterator.make_initializer(testdataset)

# with tf.Session() as sess:
#     sess.run(train_init)
# #     sess.run(val_init)
# #     sess.run(test_init)

# img_batch,Y = iterator.get_next()



In [11]:
def train_inp_fun():
    return input_fn(filenames='/home/justdial/cnn/tfrecords/tfrecordtrain',train=True)

def val_inp_fun():
    return input_fn(filenames='/home/justdial/cnn/tfrecords/tfrecordval',train=False)

def test_inp_fun():
    return input_fn(filenames='/home/justdial/cnn/tfrecords/tfrecordtrain',train=False)

In [12]:
def model_fn(features,labels,mode,params):
    X = features['image']
    net = tf.reshape(X,[-1,image_size,image_size,channels])
    
    #first convolutional layer
    net = tf.layers.conv2d(inputs = net ,
                           name = 'conv1',
                           filters = 32,
                           kernel_size = 3,
                           padding = 'same',
                           activation =  tf.nn.relu)
    
    net = tf.layers.max_pooling2d(inputs = net,
                                 pool_size=2,
                                 strides=1)
    
    #second convolutional layer
    net = tf.layers.conv2d(inputs = net ,
                           name = 'conv2',
                           filters = 32,
                           kernel_size = 3,
                           padding = 'same',
                           activation =  tf.nn.relu)
    
    net = tf.layers.max_pooling2d(inputs = net,
                                 pool_size=2,
                                 strides =1)
    
    #flatten
    net = tf.contrib.layers.flatten(net)
    
    #first fully connected layer
    net = tf.layers.dense(inputs = net,
                         name = 'fc1',
                         units = 128,
                         activation = tf.nn.relu)
    
    #second fully connected layer
    net = tf.layers.dense(inputs = net,
                         name = 'fc2',
                         units = 2, #num_classes
                         activation = tf.nn.relu)
    
    y_pred = tf.nn.softmax(logits = net)
    y_pre_cls = tf.argmax(y_pred,axis=1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        spec = tf.estimator.EstimatorSpec(mode = mode ,
                                         predictions = y_pred_cls)
        return spec
    else:
        eval_metric_ops = {"accuracy": tf.metrics.accuracy(
                    labels=labels, predictions=predictions["classes"])}
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y_pred,
                                                                     labels=labels)
        loss = tf.reduce_mean(cross_entropy)
        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate = params['learning_rate'])
            train_op = optimizer.minimize(loss=loss,
                                         global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode ,loss=loss, train_op =train_op ,accuracy=eval_metric_ops)
        else:
        
#             eval_metric_ops = {"accuracy": tf.metrics.accuracy(
#                     labels=labels, predictions=predictions["classes"])}
            #print(eval_metric_ops)
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [13]:
params={'learning_rate':1e-3}


In [14]:
model = tf.estimator.Estimator(model_fn=model_fn,
                              params = params,
                              model_dir = "./Checkpoints_example/")

In [15]:
model.train(input_fn = train_inp_fun, steps =200)

W0711 18:02:20.486825 140469486601984 deprecation.py:323] From /home/justdial/miniconda3/lib/python3.7/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0711 18:02:20.570307 140469486601984 deprecation.py:323] From <ipython-input-10-6face38b7c61>:12: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W0711 18:02:20.583705 140469486601984 deprecation.py:323] From <ipython-input-

NameError: name 'predictions' is not defined

In [ ]:
result = model.evaluate(input_fn =val_inp_fun)

In [ ]:
print(result)

In [ ]:
print('\nTest set accuracy: {}\n'.format(result))